In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
pd.set_option("display.max_columns", 50)

### Load IPL data

In [3]:
df = pd.read_csv("data/df_ipl2018_preprocessed.csv")
# is_wicket column
df["is_bowler_wicket"] = df["dismissal_kind"].isin(["caught","bowled","lbw","caught and bowled","stumped"]).values.astype(int)
del df['Unnamed: 0']

In [5]:
df.head(2)

,match_id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,is_bowler_wicket
0,638,2018,Mumbai,NaN,Mumbai Indians,Chennai Super Kings,NaN,NaN,normal,0,Chennai Super Kings,0,1,NaN,"Wankhede Stadium, Mumbai",NaN,NaN,NaN,1,Mumbai Indians,Chennai Super Kings,1,1,Rohit,NaN,Chahar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0
1,638,2018,Mumbai,NaN,Mumbai Indians,Chennai Super Kings,NaN,NaN,normal,0,Chennai Super Kings,0,1,NaN,"Wankhede Stadium, Mumbai",NaN,NaN,NaN,1,Mumbai Indians,Chennai Super Kings,1,2,Rohit,NaN,Chahar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0


In [9]:
### Number of runs scored
dfp = df.groupby(['season','venue','batting_team','batsman']).sum()["batsman_runs"].reset_index()
dfp = dfp.sort_values(['season', 'venue', 'batting_team','batsman_runs'], ascending=False)
dfp.rename(columns={"batsman":"player","batting_team":"team","batsman_runs":"runs_scored"}, inplace=True)

### Number of balls faced 
df1 = df.copy()
df1 = df1[df1['is_super_over']==0]
df1 = df1[df1['wide_runs']==0]
df1 = df1[df1['noball_runs']==0]
df1 = df1.groupby(['season','venue','batting_team','batsman']).count()["match_id"].reset_index()
df1.rename(columns={"match_id":"balls_faced","batting_team":"team","batsman":"player"}, inplace=True)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player'])).reset_index()

dfp.head()

,season,venue,team,player,runs_scored,balls_faced
0,2018,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,Williamson,29,21
1,2018,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,Y Pathan,29,33
2,2018,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,Manish Pandey,16,11
3,2018,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,Nabi,14,10
4,2018,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,Rashid Khan,6,9


In [31]:
ssr = pd.DataFrame()


for m in df["match_id"].unique():
    
    dfm = df[df["match_id"]==m]
    
    innings1_runs = df[(df["match_id"]==m) & (df["inning"]==1)].sum()["total_runs"]
    innings2_runs = df[(df["match_id"]==m) & (df["inning"]==2)].sum()["total_runs"]
    match_runs = innings1_runs + innings2_runs
    match_balls = len(dfm)
    
    batsmen = df[(df["match_id"]==m)]["batsman"].unique()
    
    for batsman in batsmen:
        batsman_balls_faced = len(df[(df["match_id"]==m) & (df["batsman"]==batsman)])
        batsman_runs_scored = df[(df["match_id"]==m) & (df["batsman"]==batsman)].sum()["batsman_runs"]
        
        nonstriker_balls_faced = len(df[(df["match_id"]==m) & (df["non_striker"]==batsman)])
        nonstriker_runs_scored = df[(df["match_id"]==m) & (df["non_striker"]==batsman)].sum()["batsman_runs"]
        
        rest_runs_scored = match_runs - batsman_runs_scored
        rest_balls_faced = match_balls - batsman_balls_faced
        
        if batsman not in ssr:
            ssr[batsman] = 0
            ssr.at["batsman_runs_scored", batsman] = batsman_runs_scored 
            ssr.at["batsman_balls_faced", batsman] = batsman_balls_faced 
            ssr.at["nonstriker_balls_faced", batsman] = nonstriker_balls_faced 
            ssr.at["nonstriker_runs_scored", batsman] = nonstriker_runs_scored 
            ssr.at["rest_runs_scored", batsman] = rest_runs_scored 
            ssr.at["rest_balls_faced", batsman] = rest_balls_faced 
        else:
            ssr.at["batsman_runs_scored", batsman] += batsman_runs_scored 
            ssr.at["batsman_balls_faced", batsman] += batsman_balls_faced 
            ssr.at["nonstriker_balls_faced", batsman] += nonstriker_balls_faced 
            ssr.at["nonstriker_runs_scored", batsman] += nonstriker_runs_scored 
            ssr.at["rest_runs_scored", batsman] += rest_runs_scored 
            ssr.at["rest_balls_faced", batsman] += rest_balls_faced 

In [36]:
dfssr = ssr.transpose().reset_index().rename(columns={"index":"batsman"})
dfssr["batsman_sr"] = dfssr["batsman_runs_scored"]/dfssr["batsman_balls_faced"] * 100
dfssr["nonstriker_sr"] = dfssr["nonstriker_runs_scored"]/dfssr["nonstriker_balls_faced"] * 100
dfssr["rest_sr"] = dfssr["rest_runs_scored"]/dfssr["rest_balls_faced"] * 100
dfssr

,batsman,batsman_runs_scored,batsman_balls_faced,nonstriker_balls_faced,nonstriker_runs_scored,rest_runs_scored,rest_balls_faced,batsman_sr,nonstriker_sr,rest_sr
0,Rohit,267.0,200.0,210.0,337.0,3306.0,2487.0,133.500000,160.476190,132.931242
1,Lewis,265.0,205.0,195.0,269.0,3000.0,2229.0,129.268293,137.948718,134.589502
2,Ishan Kishan,238.0,150.0,169.0,236.0,2659.0,2044.0,158.666667,139.644970,130.088063
3,Suryakumar Yadav,435.0,340.0,341.0,384.0,3138.0,2347.0,127.941176,112.609971,133.702599
4,Hardik Pandya,188.0,151.0,143.0,197.0,3091.0,2291.0,124.503311,137.762238,134.919249
5,Krunal Pandya,189.0,129.0,103.0,138.0,3049.0,2311.0,146.511628,133.980583,131.934228
6,Watson,367.0,245.0,264.0,342.0,3575.0,2457.0,149.795918,129.545455,145.502646
7,Rayudu,435.0,296.0,287.0,388.0,3507.0,2406.0,146.959459,135.191638,145.760599
8,Raina,313.0,242.0,228.0,334.0,3239.0,2210.0,129.338843,146.491228,146.561086
9,Dhoni,393.0,257.0,211.0,286.0,3549.0,2445.0,152.918288,135.545024,145.153374


In [40]:
dfssr[dfssr["batsman"]=="Pant"]

,batsman,batsman_runs_scored,batsman_balls_faced,nonstriker_balls_faced,nonstriker_runs_scored,rest_runs_scored,rest_balls_faced,batsman_sr,nonstriker_sr,rest_sr
21,Pant,582.0,340.0,230.0,295.0,3501.0,2394.0,171.176471,128.26087,146.240602


In [41]:
171 + 32

203

In [42]:
dfssr[dfssr["batsman"]=="Williamson"]

,batsman,batsman_runs_scored,batsman_balls_faced,nonstriker_balls_faced,nonstriker_runs_scored,rest_runs_scored,rest_balls_faced,batsman_sr,nonstriker_sr,rest_sr
59,Williamson,493.0,368.0,384.0,463.0,2783.0,2281.0,133.967391,120.572917,122.007891


In [45]:
80/40 * 100

200.0

In [44]:
120/80 * 100

150.0